**CURSO**: *Machine Learning* en Geociencias<br />
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br />
**Credits**: The content of this notebook is taken from several sources: Every effort has been made to trace copyright holders of the materials used in this notebook. The author apologies for any unintentional omissions and would be pleased to add an acknowledgment in future editions.

# 05: Análisis Exploratorio de Datos (EDA)

Luego de realizar la limpieza y ajustes sobre los datos de entrada, se debe seleccionar las mejores variables predictoras, que conformarán los *features* del modelo.

Es importante tener en cuenta en esta fase que existen diferentes métodos estadísticos que ayudan a identificar dichas variables, sin embargo el mejor método es una buena comprensión del problema y de las variables a utilizar.

A continuación se describen inicialmente los métodos para conocer cada una de las variables, y su relación con las demás variables. Y finalmente se presentan métodos para identificar de forma automática las mejores variables predictoras. Recuerde que dichos métodos no son mas que herramientas estadísticas que le pueden proporcionar informacion importante, pero la mejor selección la debe hacer el usuario considerando toda la información obtenida de las diferentes variables.

## Importar librerias

Importar los ficheros necesarios

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from osgeo import gdal
from feature_selector import FeatureSelector

%matplotlib notebook

import warnings
warnings.simplefilter("ignore")

ModuleNotFoundError: No module named 'feature_selector'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importar datos

importar en python los mapas raster

In [ ]:
ruta='/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel'

In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Aspecto.tif')
raster = file.GetRasterBand(1)
aspecto = raster.ReadAsArray()
aspecto=np.where(aspecto==-999,np.nan,aspecto)
#plt.imshow(aspecto)
print('Dimensiones de la matriz del mapa de aspecto:', aspecto.shape)
aspecto_vector=aspecto.ravel()
aspecto_vector_MenM=aspecto_vector[~np.isnan(aspecto_vector)]
print('Dimensiones del vector de aspecto:',aspecto_vector_MenM.shape)

Dimensiones de la matriz del mapa de aspecto: (1297, 1430)
Dimensiones del vector de aspecto: (910801,)


Antes de generar el dataframe con las diferentes variables, es necesario entonces importar todas las variables y realizar los ajustes que sean necesarios.

In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Buffer_Drenajes.tif')
raster = file.GetRasterBand(1)
drenajes = raster.ReadAsArray()
drenajes=np.where(drenajes==-999,np.nan,drenajes)
#plt.imshow(drenajes)
#plt.colorbar();
drenajes_vector=drenajes.ravel()
drenajes_vector_MenM=drenajes_vector[~np.isnan(drenajes_vector)]
print('Dimensiones del vector de drenajes:',drenajes_vector_MenM.shape)

Dimensiones del vector de drenajes: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Curvatura_Categorica.tif')
raster = file.GetRasterBand(1)
curvatura = raster.ReadAsArray()
curvatura = np.where(curvatura==-999,np.nan,curvatura)
#plt.imshow(curvatura)
#plt.colorbar();
curvatura_vector=curvatura.ravel()
curvatura_vector_MenM=curvatura_vector[~np.isnan(curvatura_vector)]
print('Dimensiones del vector de curvatura:',curvatura_vector_MenM.shape)

Dimensiones del vector de curvatura: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/FlujoAcumulado.tif')
raster = file.GetRasterBand(1)
flujo = raster.ReadAsArray()
flujo = np.where(flujo==-999,np.nan,flujo)
#plt.imshow(flujo)
#plt.colorbar();
flujo_vector=flujo.ravel()
flujo_vector_MenM=flujo_vector[~np.isnan(flujo_vector)]
print('Dimensiones del vector de flujo:',flujo_vector_MenM.shape)

Dimensiones del vector de flujo: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Geologia_Superficial.tif')
raster = file.GetRasterBand(1)
geologia = raster.ReadAsArray()
geologia = np.where(geologia==-999,np.nan,geologia)
#plt.imshow(geologia)
#plt.colorbar();
geologia_vector=geologia.ravel()
geologia_vector_MenM=geologia_vector[~np.isnan(geologia_vector)]
print('Dimensiones del vector de geología:',geologia_vector_MenM.shape)

Dimensiones del vector de geología: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Pendiente.tif')
raster = file.GetRasterBand(1)
pendiente = raster.ReadAsArray()
pendiente = np.where(pendiente==-999,np.nan,pendiente)
#plt.imshow(pendiente)
#plt.colorbar();
pendiente_vector=pendiente.ravel()
pendiente_vector_MenM=pendiente_vector[~np.isnan(pendiente_vector)]
print('Dimensiones del vector de pendiente:',pendiente_vector_MenM.shape)

Dimensiones del vector de pendiente: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Rugosidad.tif')
raster = file.GetRasterBand(1)
rugosidad = raster.ReadAsArray()
rugosidad = np.where(rugosidad==-999,np.nan,rugosidad)
#plt.imshow(rugosidad)
#plt.colorbar();
rugosidad_vector=rugosidad.ravel()
rugosidad_vector_MenM=rugosidad_vector[~np.isnan(rugosidad_vector)]
print('Dimensiones del vector de rugosidad:',rugosidad_vector_MenM.shape)

Dimensiones del vector de rugosidad: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/SPI.tif')
raster = file.GetRasterBand(1)
SPI = raster.ReadAsArray()
SPI = np.where(SPI==-999,np.nan,SPI)
#plt.imshow(SPI)
#plt.colorbar();
SPI_vector=SPI.ravel()
SPI_vector_MenM=SPI_vector[~np.isnan(SPI_vector)]
print('Dimensiones del vector de SPI:',SPI_vector_MenM.shape)

Dimensiones del vector de SPI: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/STI.tif')
raster = file.GetRasterBand(1)
STI= raster.ReadAsArray()
STI = np.where(STI==-999,np.nan,STI)
#plt.imshow(STI)
#plt.colorbar();
STI_vector=STI.ravel()
STI_vector_MenM=STI_vector[~np.isnan(STI_vector)]
print('Dimensiones del vector de STI:',STI_vector_MenM.shape)

Dimensiones del vector de STI: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/TWI.tif')
raster = file.GetRasterBand(1)
TWI = raster.ReadAsArray()
msk=np.where(TWI==-999.0,0,1)
TWI = np.where(TWI==-999,np.nan,TWI)
#plt.imshow(TWI)
#plt.colorbar();
TWI_vector=TWI.ravel()
TWI_vector_MenM=TWI_vector[~np.isnan(TWI_vector)]
print('Dimensiones del vector de TWI:',TWI_vector_MenM.shape)

Dimensiones del vector de TWI: (910801,)


In [ ]:
file = gdal.Open('/content/drive/My Drive/CATEDRA/MACHINE LEARNING/datos/la_miel/Inventario_MenM.tif')
raster = file.GetRasterBand(1)
inventario = raster.ReadAsArray()
inventario=np.where(msk==0,np.nan,inventario)
#plt.imshow(inventario)
inventario_vector=inventario.ravel()
inventario_vector_MenM=inventario_vector[~np.isnan(inventario_vector)]
inventario_vector_MenM.shape
print('Dimensiones del vector de inventario:',inventario_vector_MenM.shape)

Dimensiones del vector de inventario: (910801,)


Con cada uno de los vectores de las variables independientes y dependiente, se conforma un diccionario, para luego formar un DataFrame con todas las variables, y posteriormente armar un DataFrame solo con las variables predictoras (X) y un vector con la variable dependiente (y).

In [ ]:
d={'inventario':inventario_vector_MenM,'drenajes':drenajes_vector_MenM,'pendiente':pendiente_vector_MenM,'geologia':geologia_vector_MenM,'flujo':flujo_vector_MenM,'aspecto':aspecto_vector_MenM,
   'curvatura':curvatura_vector_MenM,'rugosidad':rugosidad_vector_MenM,'TWI':TWI_vector_MenM,'STI':STI_vector_MenM,'SPI':SPI_vector_MenM}
df = pd.DataFrame(d)
print(list(df.columns))
X=df.drop('inventario',axis=1)
print(X.columns)
print('Número de filas y columnas de los features:', X.shape)
y=df['inventario']
print('Número de filas del label:',y.shape)

['inventario', 'drenajes', 'pendiente', 'geologia', 'flujo', 'aspecto', 'curvatura', 'rugosidad', 'TWI', 'STI', 'SPI']
Index(['drenajes', 'pendiente', 'geologia', 'flujo', 'aspecto', 'curvatura',
       'rugosidad', 'TWI', 'STI', 'SPI'],
      dtype='object')
Número de filas y columnas de los features: (910801, 10)
Número de filas del label: (910801,)


In [ ]:
X.head()

ImportError: ignored

    drenajes  pendiente  geologia  flujo  ...  rugosidad       TWI  STI  SPI
0  61.846584  10.862183      14.0    0.0  ...   1.343179 -5.250079  0.0  0.0
1  57.008770  12.265345      14.0    0.0  ...   1.435816 -5.370969  0.0  0.0
2  52.201534  12.469252      14.0    0.0  ...   1.484001 -5.387381  0.0  0.0
3  47.434166  13.148026      14.0    0.0  ...   1.516011 -5.440150  0.0  0.0
4  42.720020  14.091524      14.0    0.0  ...   1.576045 -5.509162  0.0  0.0

[5 rows x 10 columns]

In [ ]:
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: inventario, dtype: float64

Es importante tambien crear una matriz solo con las variables predictoras continuas, ya que muchos de los métodos a utilizar solo trabajan con este tipo de variables.

In [ ]:
X_cont=X.drop(['geologia'],1)
X_array_cont=X.values
X_cont.head()

Para obtener los estadísticos básicos de todas las variables continuas se utiliza

In [ ]:
print(X_cont.describe().T)

              count          mean  ...         75%           max
drenajes   910801.0  4.413143e+01  ...   62.649822  2.437724e+02
pendiente  910801.0  2.909129e+01  ...   36.847805  7.789451e+01
flujo      910801.0  2.901423e+03  ...   26.000000  5.650063e+06
aspecto    910801.0  2.137325e+02  ...  291.331726  3.599995e+02
curvatura  910801.0  2.172012e+01  ...   31.000000  3.300000e+01
rugosidad  910801.0  2.923667e+00  ...    3.695614  3.517817e+01
TWI        910801.0  2.721776e+00  ...    3.997474  2.202815e+01
STI        910801.0  9.310124e-01  ...    1.000000  1.000000e+00
SPI        910801.0  9.327344e+09  ...  676.000000  3.192321e+13

[9 rows x 8 columns]


El problema a resolver corresponde a un método supervisado tipo clasificación, donde la variable dependiente es categórica dicotómica (la ocurrencia o no de movimientos en masa en una celda), por lo tanto es útil conocer el número de celdas con y sin MenM que permitirá entrenar el modelo.

In [ ]:
y.value_counts()

0.0    909181
1.0      1620
Name: inventario, dtype: int64

Para saber la media de cada variable independiente de acuerdo con la variable dependiente. Para esto utilizamos el DataFrame inicial (df) donde se agruparon todas las variables. 

In [ ]:
media=df.groupby('inventario').mean()
print(media)

             drenajes  pendiente   geologia  ...       TWI       STI           SPI
inventario                                   ...                                  
0.0         44.086700  29.082050  10.233438  ...  2.722892  0.931005  9.344156e+09
1.0         56.441101  34.459793   8.787037  ...  2.077408  0.935185  2.425111e+04

[2 rows x 10 columns]


Inicialmente para conocer el comportamiento bivariado de todas las variables se utiliza la matriz de scattering con Panda. Sin embargo tenga en cuenta que este método toma tiempo en ejecutarse.

In [ ]:
pd.plotting.scatter_matrix(X_cont, alpha = 0.3, figsize = (14,10), diagonal='kde');

La librería de *Seaborn* es similar a Matplotlib, sin embargo presenta gráficas con mejores diseños. La matriz de scattering utilizando *Seaborn* se genera de la siguiente manera, y de forma similar al caso anterior toma tiempo.

In [ ]:
sns.pairplot(df, hue='inventario');

In [ ]:
sns.boxplot(data=df, x='inventario', y='pendiente');

In [ ]:
sns.histplot(data=df, x='inventario', hue='pendiente', bins=50);

In [ ]:
sns.histplot(x='inventario', data=df, hue='pendiente', bins=len(df), stat="density",
             element="step", fill=False, cumulative=True, common_norm=False);
plt.title("Cumulative distribution function");

## Análisis univariado

Para generar el histograma de una sola variable se puede utilizar el siguiente código con el método *hist*.:

In [ ]:
X.pendiente.hist()
plt.title('Histograma de Pendiente')
plt.xlabel('Pendiente')
plt.ylabel('Frecuencia');

ImportError: ignored

Con la librearía *Seaborn* se puede generar el *displot* de las diferentes variables.

In [ ]:
sns.distplot(X['aspecto'],color='g');

KeyError: ignored

Para generar el histograma de las variables continuas.

In [ ]:
X_cont.plot(kind='density', subplots=True, layout=(3,3), sharex=False, figsize=(10, 4));

ImportError: ignored

Para generar el diagrama de caja de cada variable:

In [ ]:
pendiente=X['pendiente']
plt.boxplot(pendiente);

KeyError: ignored

In [ ]:
sns.boxplot(x='pendiente', data=X);

El diagrama tipo Violin brinda información similar al *boxplot*, sin embargo en algunos casos puede brindar información adicional sobre la distribución de los datos.

In [ ]:
sns.violinplot(x='pendiente', data=X);

## Análisis Bivariado

El análisis bivariado permite identificar asociación o correlación entre diferentes variables. Se utilizan las siguientes herramientas.

In [ ]:
sns.jointplot(x='pendiente', y='TWI', data=X, kind='scatter');

In [ ]:
sns.jointplot(x='pendiente', y='flujo', data=X, kind='kde', color='g');

Un análisis muy importante es la distribución de cada variable independiente en función de la variable dependiente. Distribuciones diferentes permite inferir que dicha variable puede ser buena predictora.

Inicialmente se construyen dos nuevos dataFrames filtrando entre celdas donde la variabel independiente es 1 y celdas donde es 0, en este caso CON y SIN movimientos en masa.

In [ ]:
data_sin=df[(df['inventario']==0)]
data_con=df[(df['inventario']==1)]

Con estos dos DataFrames se pueden comparar las variables indepependientes en funcion de y.

In [ ]:
fig, ax = plt.subplots()
data_sin['pendiente'].plot.kde(ax=ax, label='Sin MenM')
data_con['pendiente'].plot.kde(ax=ax, label='Con MenM')
ax.set_xlim(0,90)
ax.set_xlabel('Pendiente (grados)', color='k', size=12)
ax.set_ylabel('Densidad', color='k', size=12)
ax.legend(loc=1, fontsize=10)
ax.tick_params('y', colors='k', labelsize= 10)

## Análisis Multivariado

El análisis multivariado permite analizar tres o mas variables.

In [ ]:
sns.scatterplot(x="curvatura", y="pendiente", hue="inventario", data=df);

Una etapa fundamental en el análisis multivariado de los datos es evaluar la correlación entre ellos. Para lo cual existen diferentes herramientas.

Una de ellas es utilizando la librería *feature Selector* para todas las variables. Para lo cual es necesario inicialmente instanciar el método, y luego correr la función *indetify_collinear*, donde se debe precisar un umbral de correlación.

In [ ]:
fs = FeatureSelector(data = X, labels = y)

In [ ]:
fs.identify_collinear(correlation_threshold=0.5)
correlated_features = fs.ops['collinear']
correlated_features

In [ ]:
fs.plot_collinear(plot_all=True)

Para obtener una tabla con las correlaciones se utiliza:

In [ ]:
fs.record_collinear.head()

La librearía *statsmodel* también brinda herramientas para identificar y plotear la matriz de correlación.

In [ ]:
MatCorre=DataFrame(X.corr())
smg.plot_corr(MatCorre, xnames=list(MatCorre.columns));

Y la libraría *Seaborn* también tiene una función para plotear la matriz, donde a diferencia de las demas marca el valor de la correlación en cada celda.

In [ ]:
plt.figure(figsize=(12,10))
cor = X.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()